In [198]:
# Cluster via K Means, DBSCAN, Spectral Clustering

In [199]:
# Graph and save V-measure, ARI, Adjusted Mutual Information

In [557]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline
import csv
import pandas as pd
from pandas import DataFrame
def load_data(file_name: str = "Data.csv" , debug: bool = False ) -> DataFrame:
    
    try:
        with open(file_name, mode ='r')as file:
            csv_file = csv.reader(file)
        df = pd.read_csv(file_name)
        if debug:
            print("Dataframe after loading data")
            print(df.head(5))
    except:
        print("Could Not Load File")
   
    if debug:
        print("Loading Data Completed")
    return df

In [581]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import RobustScaler
import numpy as np
# Vectorizing Method: 
def vectorize_data(df, v_method: int = 0, debug:bool = False) -> DataFrame:
    
    all_vectorized_dfs = []

    # Iterate over each column except the first one
    for column in df.columns[3:]:
        if v_method == 0:
            vectorizer = TfidfVectorizer(stop_words='english')
        elif v_method == 1:
            vectorizer = CountVectorizer(stop_words='english')

        else:
            raise ValueError(f"Vectorizing method '{v_method}' is not supported.")
        transformed_data = vectorizer.fit_transform(df[column])

        if debug:
            print(f"Shape after vectorizing column '{column}':", transformed_data.shape)

        # Convert the sparse matrix to a DataFrame
        vectorized_df = pd.DataFrame(transformed_data.toarray(), columns=[f"{column}_{feature}"
                        for feature in vectorizer.get_feature_names_out()])
        all_vectorized_dfs.append(vectorized_df)


    # Concatenate all the vectorized DataFrames along the columns
    df_vectorized = pd.concat(all_vectorized_dfs, axis=1)

    # Append the first column of the original DataFrame
    df_combined = pd.concat([df.iloc[:, [0]], df_vectorized], axis=1)

    if debug:
        print("Final DataFrame shape after vectorizing all columns:", df_final.shape)
    df_combined = df_combined.replace([np.inf, -np.inf], np.nan)
    df_filled = df_combined.fillna(-1)
    df_names = df_filled.drop("Character",axis='columns')
    df_filled.drop("Character",axis='columns', inplace=True)
    # Fit and transform the data
    scaler = RobustScaler()
    scaled_data = scaler.fit_transform(df_filled)

    # Convert the scaled data back to a DataFrame
    df_final = pd.DataFrame(scaled_data, columns=df_filled.columns)
    return df_final,df_names


In [590]:
import pandas as pd
from sklearn.decomposition import PCA, FastICA, KernelPCA
from sklearn.neural_network import BernoulliRBM
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, v_measure_score, silhouette_score

def data_reduction(file_name: str,df: pd.DataFrame, method: int = 0, clusters: int = 3, debug: bool = False) -> pd.DataFrame:
    if debug:
        print("Starting data reduction")
    temp_df = df
    char_names = df.drop(df.columns[0], axis=1, inplace=True)
    #print(char_names)
    #print(temp_df.head(5))
    # Select the reduction method
    if method == 0:
        reduced_data = df 
        reduction_method = "Double-RBM"
    elif method == 1:
        reducer = PCA(n_components=2)
        reduction_method = "PCA"
    elif method == 2:
        reducer = FastICA(n_components=2)
        reduction_method = "ICA"
    elif method == 3:
        feats = len(df.columns)
        layer_one = round(feats,2)
        if debug:
            print(f"Layer Size: {layer_one}")
        reducer = BernoulliRBM(n_components=layer_one)
        reducer = BernoulliRBM(n_components=2)
        reduction_method = "Double-RBM"
    else:
        raise ValueError("Unknown reduction method specified")
    if method != 0:
        # Apply the reduction
        reduced_data = reducer.fit_transform(temp_df)
    
    if debug:
        print(f"Reduction method: {reduction_method}")
        print("Reduced data shape:", reduced_data.shape)
    
    # Perform clustering
    clustering_algorithms = {
        'K-Means': KMeans(n_clusters=clusters, random_state=42,n_init='auto'),
        'Spectral': SpectralClustering(n_clusters=clusters, random_state=42, affinity='nearest_neighbors')
    }

    evaluations = []

    for name, algorithm in clustering_algorithms.items():
        labels = algorithm.fit_predict(reduced_data)
        print(f"Clustering method: {name}")
        temp_df[f'Cluster_{name}'] = labels
        temp_df.to_csv(f"{file_name} meth_{reduction_method} c_{clusters}.csv")
        sil, ari, ami, v_measure = evaluate_clustering(temp_df, labels)
        evaluations.append((file_name,reduction_method, name,clusters, sil, ari, ami, v_measure))
    

   
    return evaluations

def evaluate_clustering(x, y):
    # Shuffle the y labels to create random labels
    shuffled_labels = np.random.permutation(y)
    
    # Compute silhouette score with the provided labels
    sil = silhouette_score(x, y)
    
    # Compute ARI, AMI, and V-measure with shuffled labels
    ari = adjusted_rand_score(y, shuffled_labels)
    ami = adjusted_mutual_info_score(y, shuffled_labels)
    v_measure = v_measure_score(y, shuffled_labels)
    
    return sil, ari, ami, v_measure


In [588]:
def __main__(file_name: str = "data.csv",debug: bool = False)-> int:
    df = load_data(file_name,False)
    v = [0,1]
    r = [0,1,2]
    clusters = [2,3,4,5,6,7,8]
    results = []
    for v_ind in v:
        v_df,names = vectorize_data(df,v_ind,False)
        for red_ind in r:
            for c in clusters:
                test_results = data_reduction(file_name[:-4],v_df,red_ind,c)
                for t in test_results:
                    results.append(t)
    final_results = pd.DataFrame(results,columns=["Dataset", "Reduction_method","Clustering Method",
                                                 'clusters', 'sil', 'ari', 'ami', 'v_measure'])
    final_results.to_csv(f"{file_name[:-4]} Test Results.csv")
    return 1;

In [589]:
debug = True
__main__("Smash 64 v1.csv",debug)

C:\Users\jaylo\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:1440: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\jaylo\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:1440: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Clustering method: K-Means
Clustering method: Spectral
Clustering method: K-Means


C:\Users\jaylo\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:1440: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\jaylo\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:1440: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Clustering method: Spectral
Clustering method: K-Means


C:\Users\jaylo\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:1440: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\jaylo\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:1440: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


...

C:\Users\jaylo\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:1440: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\jaylo\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:1440: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Clustering method: Spectral
Clustering method: K-Means


C:\Users\jaylo\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:1440: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\jaylo\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:1440: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Clustering method: Spectral


1

### 